In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import logging
import numpy as np
import socket
hostname = socket.gethostname()

import sys
import mxnet as mx

In [4]:
stack = 'MD589'

In [16]:
# for sec in range(154, 169):
for sec in [168]:

    t = time.time()

    print sec

    data_dir = '/home/yuncong/CSHL_data_patches/%(stack)s_byROI/%(sec)04d/roi1'%{'stack':stack,'sec': sec}

    
    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}
    
    img_filenames = sorted(os.listdir(data_dir))
    with open('/home/yuncong/CSHL_data_patches/mxnet_iterators/%(dataset)s_test.lst' % \
              {'dataset': dataset}, 'w') as lst_file:
        for index, img_filename in enumerate(img_filenames):
            lst_file.write('%s\t0\t%s\n'%(index, img_filename))
            
    os.system(os.environ['MXNET_DIR']+'/bin/im2rec /home/yuncong/CSHL_data_patches/mxnet_iterators/%(dataset)s_test.lst \
%(data_dir)s/ /home/yuncong/CSHL_data_patches/mxnet_iterators/%(dataset)s_test.rec' % {'dataset': dataset, 'data_dir': data_dir})
    
    print time.time() - t
    
# on mbp, scanning 1000 images take 1.8s
# on gordon, scanning 1000 image, 3.4s
# on bs workstation, scanning 1000 patches, 4.3s

168
81.4891009331


In [5]:
prefix = 'inception-stage1'
n_iter = 10

model_dir = '/home/yuncong/mxnet_models/' 
init_model = mx.model.FeedForward.load(os.path.join(model_dir, prefix), n_iter, ctx=mx.gpu())

In [6]:
import time

In [7]:
pred_result_dir = '/home/yuncong/CSHL_data_patches/%(stack)s_roi1_predictionResults/' % {'stack': stack}
if not os.path.exists(pred_result_dir):
    os.makedirs(pred_result_dir)

In [8]:
logging.basicConfig(level=logging.DEBUG)

# for sec in range(154, 169):
# for sec in range(159, 169):

d = {}

for sec in [168]:

    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

    # if the dataset has more than one batch's amount of data, then you can see effect of multithreading enabled by openblas


    for batch_size in [16, 32, 64, 128, 256, 512, 1024]:
#     batch_size = 256

#         test_iter.reset()

        test_iter = mx.io.ImageRecordIter(
            path_imgrec='/home/yuncong/CSHL_data_patches/mxnet_iterators/%(dataset)s_test.rec' % {'dataset': dataset},
            batch_size=batch_size,
            data_shape=(3, 224, 224),
            mean_img=os.path.join(model_dir, 'mean_224.nd'),
        )

        t = time.time()

        probs = init_model.predict(test_iter)

        # on Gordon, predicting 1 patch takes .5 s; OPENBLAS_NUM_THREADS=16 but CPUs are not fully utilized
        # on mbp, predicting 16 patches takes 1 s

        print time.time() - t

        d[batch_size] = time.time() - t
    
    
    # prediction_map = zip(np.argmax(probs, axis=1), map(lambda x: int(x[:-4]), img_filenames))
#     np.save(os.path.join(pred_result_dir, '%(dataset)s_test_prediction_results.npy'% {'dataset': dataset}), 
#             probs)

    # 900 sec = 15 min for 20k patches
    # 142 sec = 2.5 min for 20k patches on Titan X, batch size = 16
    # 86 sec, if batch size = 128
    # 56 sec if batch size = 
    

98.6503019333


MXNetError: [22:31:54] src/ndarray/ndarray.cc:227: Check failed: from.shape() == to->shape() operands shape mismatch

In [27]:
d

{16: 62.01036095619202,
 32: 51.897693157196045,
 64: 51.10006403923035,
 128: 51.13325595855713,
 256: 50.75234508514404,
 512: 50.51603603363037,
 1024: 51.73735594749451}